In [1]:
import os
import numpy as np
from PIL import Image

from src.controller.ml_controller import MLController

################## PATH ##################
LFW_DATASET_PATH = r"..\data\dataset-lfw_reconstructed"
DB_PATH = r"..\data\gui_database.db"
ML_OUTPUT = r"..\data\ml_models"

# Import Data

### Noised DB dataset

In [2]:
mlc = MLController(db_path=DB_PATH, ml_output=ML_OUTPUT)
print(mlc.X.shape)

(511, 100, 100, 3)


### Noised LFW dataset

In [3]:
from src.modules.data_loader import load_anonymized_images_flat
os.makedirs(LFW_DATASET_PATH, exist_ok=True)


X, y, label_encoder = load_anonymized_images_flat(
    data_dir=LFW_DATASET_PATH,
    img_width=100,
    img_height=100,
    color_mode='grayscale'
)

if not X.shape and not y.shape and not label_encoder:
    raise ValueError('Critical error while loading data. Script stopped..')
print(f"\n(nb_image, width, height, channels) : {X.shape}")

mlc = MLController(db_path=DB_PATH, ml_output=ML_OUTPUT, data=(X, y, label_encoder))

Chargement des images depuis : ..\data\dataset-lfw_reconstructed
Format attendu : 100x100, mode: grayscale
Trouvé 1240 fichiers PNG.
Chargement terminé. 1240 images traitées, 0 fichiers ignorés.
Shape final des données (X): (1240, 100, 100, 1)
1240 labels originaux encodés en 62 classes numériques.

(nb_image, width, height, channels) : (1240, 100, 100, 1)


# Train

In [4]:
mlc.prepare_data()
print(mlc.X_train.shape)

Division des données : test_size=0.2, validation_size=0.15
Taille Test: 248 échantillons
Taille Entraînement: 806 échantillons
Taille Validation: 186 échantillons
(806, 100, 100, 1)


In [5]:
mlc.create_model()

Construction du modèle CNN simple avec input_shape=(100, 100, 1) et num_classes=62
Modèle CNN simple construit.


Model: "simple_cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_image (InputLayer)        │ (None, 100, 100, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_1 (Conv2D)                │ (None, 100, 100, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn1_1 (BatchNormalization)      │ (None, 100, 100, 32)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_2 (Conv2D)                │ (None, 100, 100, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn1_2 (BatchNormalization)      │ (None, 100, 100, 32)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1 (MaxPooling2D)            │ (None, 50, 50, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop1 (Dropout)                 │ (None, 50, 50, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2_1 (Conv2D)                │ (None, 50, 50, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn2_1 (BatchNormalization)      │ (None, 50, 50, 64)     │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2_2 (Conv2D)                │ (None, 50, 50, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn2_2 (BatchNormalization)      │ (None, 50, 50, 64)     │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool2 (MaxPooling2D)            │ (None, 25, 25, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop2 (Dropout)                 │ (None, 25, 25, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 40000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense1 (Dense)                  │ (None, 128)            │     5,120,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_dense1 (BatchNormalization)  │ (None, 128)            │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop_dense1 (Dropout)           │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_softmax (Dense)          │ (None, 62)             │         7,998 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,194,398 (19.82 MB)

 Trainable params: 5,193,758 (19.81 MB)

 Non-trainable params: 640 (2.50 KB)

In [ ]:
result = mlc.train_model()
print(f"--- Training Script Completed in {mlc.duration:.2f} secondes ---")
for key, val in result.items(): print(f"\n{key} : {val}")


Epoch 1: val_accuracy improved from -inf to 0.01613, saving model to ..\data\ml_models/trained\simple_cnn_lfw_anony_v1.h5



Epoch 2: val_accuracy did not improve from 0.01613

Epoch 3: val_accuracy did not improve from 0.01613

Epoch 4: val_accuracy did not improve from 0.01613

Epoch 5: val_accuracy did not improve from 0.01613

Epoch 6: val_accuracy did not improve from 0.01613

Epoch 7: val_accuracy did not improve from 0.01613

Epoch 8: val_accuracy did not improve from 0.01613

Epoch 9: val_accuracy improved from 0.01613 to 0.02151, saving model to ..\data\ml_models/trained\simple_cnn_lfw_anony_v1.h5



Epoch 10: val_accuracy improved from 0.02151 to 0.02688, saving model to ..\data\ml_models/trained\simple_cnn_lfw_anony_v1.h5



Epoch 11: val_accuracy improved from 0.02688 to 0.04301, saving model to ..\data\ml_models/trained\simple_cnn_lfw_anony_v1.h5



Epoch 12: val_accuracy improved from 0.04301 to 0.06452, saving model to ..\data\ml_models/trained\simple_cnn_lfw_anony_v1.h5


...


Epoch 21: val_accuracy improved from 0.93011 to 0.94624, saving model to ..\data\ml_models/trained\simple_cnn_lfw_anony_v1.h5



Epoch 22: val_accuracy improved from 0.94624 to 0.96237, saving model to ..\data\ml_models/trained\simple_cnn_lfw_anony_v1.h5



Epoch 23: val_accuracy improved from 0.96237 to 0.97312, saving model to ..\data\ml_models/trained\simple_cnn_lfw_anony_v1.h5



Epoch 24: val_accuracy improved from 0.97312 to 0.98387, saving model to ..\data\ml_models/trained\simple_cnn_lfw_anony_v1.h5



Epoch 25: val_accuracy did not improve from 0.98387

Epoch 26: val_accuracy did not improve from 0.98387

Epoch 27: val_accuracy did not improve from 0.98387


# Predict noised image

In [ ]:
user = 42
image_path = f"../data/dataset-lfw_reconstructed/reconstructed_{user}_2.png"
image = np.array(Image.open(image_path))

mlc = MLController(db_path=DB_PATH, ml_output=ML_OUTPUT)
result = mlc.predict_image(image)
predicted_label, prediction_confidence = result
print(f"  - Predicted Identity (Subject ID) : {predicted_label}")
print(f"  - Trust : {prediction_confidence:.4f} ({prediction_confidence*100:.2f}%)")